# *checkpoint_schedule* application: Adjoint-Based Gradient with the Burger's Equation

This example shows adjoint-based gradient computation using the *checkpointing_schedules* package. We initially define the adjoint-based gradient problem and then present the forward and adjoint solvers prescribed by the *checkpointing_schedules* package.

## Defining the application

Let us consider a one-dimensional (1D) problem aiming to compute the gradient/sensitivity of the kinetic energy at a time $t=\tau$ with respect to an initial condition $u_0$. Then, our functional can be expressed as:
$$
I(u(x, \tau, u_0)) = \int_\Omega \frac{1}{2} u(x, \tau, u_0)u(x, \tau, u_0) \, dx.
\tag{1} 
$$

In the current case, the velocity variable $u = u(x, t)$ is governed by the 1D viscous Burgers equation, a non-linear equation for the advection and diffusion of momentum:

$$
\frac{\partial u}{\partial t} + u \frac{\partial u}{\partial x} - \nu \frac{\partial^2 u}{\partial x^2} = 0.
\tag{2}
$$

Here, $x \in [0, L]$ is the space variable, and $t \in \mathbb{R}^{+}$ represents the time variable. The boundary condition is $u(0, t) = u(L, t) = 0$, where $L$ is the length of the 1D domain. The initial condition is given by $u_0 = \sin(\pi x)$.

The control parameter is the initial condition $u_0$. Hence, the objective is to compute the adjoint-based gradient of the functional $I(u_0)$ with respect to $u_0$.

This example uses the discrete adjoint formulation, meaning the adjoint system is from the discrete forward PDE system.

## Adjoint-based gradient
We wain to compute the sensitivity/gradient $\mathrm{d}\widehat{I}/\mathrm{d}u_0$, where $\widehat{I} = I(u_0)$ is the reduced functional. On applying the chain rule, we have the below expression on considering the functional $I$ as in Eq. (1).
$$\frac{\mathrm{d}\widehat{I}}{\mathrm{d}u_0} = \frac{\partial I}{\partial u} \frac{\mathrm{d}u}{\mathrm{d}u_0}. \tag{3}$$
Computing $\partial J/\partial u$ is straightforward since the functional (Eq. (1)) is written in terms of the solution variable $u$. In contrast, the Jacobian $\mathrm{d}u/\mathrm{d}u_0$ involves an expensive computational operation that is avoided by using the adjoint method.

In summary, the adjoint system is defined by using the relation of the functional $I$ with the forward equation $F(u, u_0)$. Taking the total derivative of $F(u, u_0)$ with respect to $u_0$ yields a relationship for the solution Jacobian $\mathrm{d}u/\mathrm{d}u_0$:
$$\frac{\mathrm{d}}{\mathrm{d}u_0} F(u, u_0) = \frac{\partial F(u, u_0)}{\partial u} \frac{\mathrm{d}u}{\mathrm{d}u_0} + \frac{\partial F(u, u_0)}{\partial u_0} = 0 \\
 \implies \frac{\mathrm{d}u}{\mathrm{d}u_0} = - \left(\frac{\partial F(u, u_0)}{\partial u}\right)^{-1} \frac{\partial F(u, u_0)}{\partial u_0}.$$
Substituting $\mathrm{d}u/\mathrm{d}u_0$ into Eq. (3), we have:
$$\frac{\mathrm{d}\widehat{I}}{\mathrm{d}u_0} = - \frac{\partial I}{\partial u} \left(\frac{\partial F(u, u_0)}{\partial u}\right)^{-1} \frac{\partial F(u, u_0)}{\partial u_0}.$$
We can avoid the inverse $\left(\frac{ \partial F(u, u_0)}{\partial u}\right)^{-1}$ computation by taking the adjoint (Hermitian transpose) of the above equation:
$$\frac{\mathrm{d}\widehat{I}}{\mathrm{d}u_0}^* = - \frac{\partial F}{\partial u_0}^* \frac{\partial F}{\partial u}^{-*} \frac{\partial I}{\partial u}^{*}$$

Next, we define:
$$\lambda = \left(\frac{\partial F(u, u_0)}{\partial u}\right)^{-*} \frac{\partial I}{\partial u}^* 
  \implies  \left(\frac{\partial F(u, u_0)}{\partial u}\right)^{*} \lambda = \frac{\partial I}{\partial u}^*,$$
which is the adjoint system. Finally, we obtain the following expression for the adjoint-based gradient computation:
$$\frac{\mathrm{d}\widehat{I}}{\mathrm{d} u_0}^* = - \frac{\partial F}{\partial u_0}^* \lambda.$$
In this current case, Burger's equation is written in weak form and discretized in time with the backward finite difference method, we have:
$$ F(u^{n+1}, u^n, v) = \int_{\Omega} u^{n + 1} \cdot v - u^{n}  \cdot v + \Delta t \left(C(u^{n + 1}) u^{n + 1} \cdot v + \frac{\partial u^{n + 1}}{\partial x}  \cdot \frac{\partial v}{\partial x}\right) \, dx = 0 \quad \forall u, v\in V \tag{4},$$
where $v \in V$ is an arbitrary test function. The Jacobian of $F(u^{n+1}, u^n, v)$ is obtained through the Gateaux derivative:
$$J(u^{n+1}, u^{n}, v, \delta u) = \lim_{\epsilon \to 0} \frac{F(u^{n+1} + \epsilon \delta u, u^{n}, v) - F(u^{n+1}, u^{n}, v)}{\epsilon} \\
\implies J(u^{n+1}, u^{n}, v, \delta u) = \int_{\Omega} \delta u \cdot v + \Delta t \left(C(u^{n + 1}) \delta u\cdot v + C(\delta u) u^{n+1} \cdot v + \frac{\partial \delta u}{\partial x} \frac{\partial v}{\partial x}\right) \, dx. \tag{5}$$
$\delta u$ is a directional perturbation. The adjoint equation then takes the form:
$$J(u^{n+1}, u^{n}, v, \delta u)^{*} \lambda^{n+1} = \frac{\partial I}{\partial u^{n+1}}^{*}.$$

For additional details on the adjoint method, including a discussion on its cost-effectiveness compared to the direct computation of the Jacobian of the forward solution with respect to the control parameter, please refer to the following [reference](https://www.dolfin-adjoint.org/en/latest/documentation/maths/3-gradients.html).

## Burger's equation solvers
We define a `BurgersEquation` class to compute the `forward` and `adjoint` solvers. In addition, the `BurgersEquation` class defines a `copy_data` method that copies the data from one storage type to another, and an `adjoint_initial_condition` method that initializes the adjoint.

The Burger's equation is discretised using the Finite Element Method (FEM). We use continuous first-order Lagrange basis functions to discretise the spatial domain.

After discretising, the forward system is written in the following form:
$$M \mathbf{U}^{n + 1} - M \mathbf{U}^{n} + \Delta t \left(C(\mathbf{U}^{n + 1}) \mathbf{U}^{n + 1} - K \mathbf{U}^{n + 1}\right) = 0, \tag{6}$$
where $M$ and $K$ are the mass and stiffness, respectively. $U$ represents the solution vector. The matrices $C$ is obtained from the advection term. The superscript $n$ denotes the time step, and $\Delta t$ is the time step size. The solution vector $\mathbf{U}^{n + 1}$ is obtained by solving the non-linear forward system using the Newton method. The Jacobian of the forward system has the form:
$$J = M + \Delta t \left(J_C(\mathbf{U}^{n + 1}) - K \right) \tag{7},$$
where $J_C$ is the Jacobian of the advection term. The adjoint system is then written as: 
$$J^{T} \Lambda^{n + 1} = \frac{\partial I}{\partial \mathbf{U}^{n + 1}}^{T}, \tag{8}$$
where $\Lambda$ is the adjoint solution vector.

In [3]:
import numpy as np
import os
from scipy.sparse.linalg import spsolve
from scipy.sparse import lil_matrix
from scipy.optimize import newton
import copy


class BurgersEquation:
    """A solver for the forward and adjoint one dimensional Burger's equation.

    Parameters
    ----------
    model : dict
        The model parameters containing the essential information to solve
        the Burger's equation.
    init_condition : array
        The initial condition used to solve the forward Burger's equation.
    mesh : array
        The spatial mesh.
    """
    def __init__(self, model, forward_initial_condition, mesh):
        self.model = model
        self.mesh = mesh
        self.forward_work_memory = {StorageType.WORK: {}}
        self.forward_work_memory[StorageType.WORK][0] = copy.deepcopy(forward_initial_condition)
        self.forward_final_solution = None
        self.adjoint_work_memory = {StorageType.WORK: {}}
        self.restart_forward = {StorageType.RAM: {}, StorageType.DISK: {}}
        self.adjoint_dependency = {StorageType.WORK: {}, StorageType.RAM: {}, StorageType.DISK: {}}
        self.mode = "forward"

    def forward(self, n0, n1, storage=None, write_adj_deps=False, write_ics=False):
        """Advance the forward solver.

        Parameters
        ----------
        n0 : int
            Initial time step.
        n1 : int
            Final time step.
        storage : StorageType, optional
            The storage type, which can be StorageType.RAM, StorageType.DISK,
            StorageType.WORK, or StorageType.NONE.
        write_adj_deps : bool, optional
            Whether the adjoint dependency data will be stored.
        write_ics : bool, optional
            Whether the forward restart data will be stored.
        """
        M = self._mass_matrix()
        K = self._stiffness_matrix()
        # Get the initial condition
        u = self.forward_work_memory[StorageType.WORK][n0]
        del self.forward_work_memory[StorageType.WORK][n0]
        if write_ics:
            self.restart_forward[storage][n0] = u.copy()
        u_new = copy.deepcopy(u)
        n1 = min(n1, self.model["max_n"])
        step = n0
        while step < n1:
            if write_adj_deps:
                self.adjoint_dependency[storage][step] = copy.deepcopy(u)
            def _residual(u_new):
                C = self._convection_matrix(u_new)
                residual = (M + self.model["dt"] * (- K + C)).dot(u_new) - M.dot(u)
                residual[0] = residual[self.model["nx"] - 1] = 0
                return residual
            u_new = self._solve_newton(u, _residual)
            u = copy.deepcopy(u_new)
            step += 1
        if step == self.model["max_n"]:
            self.forward_final_solution = copy.deepcopy(u_new)
        if (not write_adj_deps 
           or (self.mode == "forward" and step < (self.model["max_n"]))
        ):
            self.forward_work_memory[StorageType.WORK][step] = copy.deepcopy(u_new)

    def adjoint(self, n0, n1, clear_adj_deps):
        """Advance the adjoint solver.

        Parameters
        ---------
        n0 : int
            Initial time step.
        n1 : int
            Final time step.
        clear_adj_deps : bool
            If `True`, the adjoint dependency data will be cleared.
        """
        u = np.zeros(self.model["nx"])
        if n1 < self.model["max_n"]:
            u = self.adjoint_work_memory[StorageType.WORK][n1]
            del self.adjoint_work_memory[StorageType.WORK][n1]
        step = n1
        M = self._mass_matrix()
        while step > n0:
            A = self._jacobian(self.adjoint_dependency[StorageType.WORK][step - 1]).T
            if step == self.model["max_n"]:
                u_forward = copy.deepcopy(self.forward_final_solution)
                d = M.T.dot(u_forward)
            else:
                d = M.T.dot(u)
            d[0] = d[-1] = 0
            u[1:-1] = spsolve(A[1:-1, 1:-1], d[1:-1])
            if clear_adj_deps:
                del self.adjoint_dependency[StorageType.WORK][step-1]
            step -= 1
        # Apply the adjoint boundary condition
        u[0] = u[-1] = 0
        self.adjoint_work_memory[StorageType.WORK][step] = copy.deepcopy(u)

    def gradient(self):
        """Compute the adjoint-based gradient.

        Returns
        -------
        array
            The gradient.
        """
        M = self._mass_matrix()
        M[0, :] = M[self.model["nx"] - 1, :] = 0
        M[:, 0] = M[:, self.model["nx"] - 1] = 0
        M[0, 0] = M[self.model["nx"] - 1, self.model["nx"] - 1] = 1
        return self.adjoint_work_memory[StorageType.WORK][0]
    
    def functional(self):
        """Compute the functional.
        """
        return 0.5 * sum(self.forward_final_solution ** 2)
    

    def copy_data(self, step, from_storage, to_storage, move=False, single_storage=False):
        """Copy data from one storage to another.

        Parameters
        ----------
        step : int
            The time step.
        from_storage : StorageType
            The storage type from which the data will be copied.
        to_storage : StorageType
            The storage type to which the data will be copied.
        move : bool, optional
            Whether the data will be moved or not. If `True`, the data will be
            removed from the `from_storage`.
        """
        if to_storage == StorageType.WORK:
            if single_storage:
                self.adjoint_dependency[StorageType.WORK][step] = self.adjoint_dependency[from_storage][step]
            else:
                self.forward_work_memory[StorageType.WORK][step] = self.restart_forward[from_storage][step]
        else:
            self.restart_forward[to_storage][step] = self.restart_forward[from_storage][step]
        if move:
            if single_storage:
                del self.adjoint_dependency[from_storage][step]
            else:
                del self.restart_forward[from_storage][step]
    
    def _jacobian(self, u):
        M = self._mass_matrix()
        K = self._stiffness_matrix()
        C = self._jacobian_advection(u)
        Jac = M + self.model["dt"] * (- K + C)
        Jac[0, :] = Jac[-1, :] = 0
        Jac[0, 0] = Jac[self.model["nx"] - 1, self.model["nx"] - 1] = 1
        return Jac
    
    def _solve_newton(self, u_prev, residual, tol=1e-6, max_iter=50):
        i = 0
        delta_u = 1
        u_new = copy.deepcopy(u_prev)
        while i < max_iter:
            Jac = self._jacobian(u_new)
            delta_u = spsolve(Jac, residual(u_new))
            u_new -= delta_u
            i += 1
            if np.linalg.norm(delta_u) < tol:
                break
        if i == max_iter:
            print("Newton's method did not converge in {} iterations.".format(max_iter))
        return u_new

    def _mass_matrix(self):
        num_nodes = self.model["nx"]
        M = lil_matrix((num_nodes, num_nodes))
        M[0, 0] = M[num_nodes - 1, num_nodes - 1] = 1/3
        M[0, 1] = M[num_nodes - 1, num_nodes - 2] = 1/6
        for i in range(1, num_nodes - 1):
            M[i, i - 1] = M[i, i + 1] = 1/6
            M[i, i] = 2/3
        return M
    
    def _stiffness_matrix(self):
        num_nodes = self.model["nx"]
        h = self.model["lx"] / self.model["nx"] 
        # 1D mesh is uniform. Thus, the mesh spacing is constant.
        b = self.model["nu"] / (h ** 2)
        K = lil_matrix((num_nodes, num_nodes))
        K[0, 0] = K[num_nodes - 1, num_nodes - 1] = - b
        K[0, 1] = K[num_nodes - 1, num_nodes - 2] = + b
        for i in range(1, num_nodes - 1):
            K[i, i - 1] = K[i, i + 1] = b
            K[i, i] = - 2 * b
        return K
    
    def _convection_matrix(self, u):
        num_nodes = self.model["nx"]
        # 1D mesh is uniform. Thus, the mesh spacing is constant.
        h = self.model["lx"] / self.model["nx"] 
        C = lil_matrix((num_nodes, num_nodes))
        C[0, 0] = - (1/3 * u[0] / h + 1/6 * u[1] / h)
        C[num_nodes - 1, num_nodes - 1] = 1/3 * u[num_nodes - 1] / h + 1/6 * u[num_nodes - 2] / h
        C[0, 1] = - C[0, 0]
        C[num_nodes - 1, num_nodes - 2] = - C[num_nodes - 1, num_nodes - 1]
        for i in range(1, num_nodes - 1):
            C[i, i - 1] = - (1/6 * u[i - 1] / h + 1/3 * u[i] / h)
            C[i, i] = (1/6 * u[i - 1] / h - 1/6 * u[i + 1] / h)
            C[i, i + 1] = (1/6 * u[i + 1] / h + 1/3 * u[i] / h)
        return C
    
    def _jacobian_advection(self, u):
        N = self.model["nx"]
        # 1D mesh is uniform. Thus, the mesh spacing is constant.
        h = self.model["lx"] / self.model["nx"] 
        jac = lil_matrix((N, N))
        jac[0, 0] = -2 * u[0] / 3 + u[1] / 6
        jac[0, 1] = u[0] / 6 + u[1] / 3
        jac[N - 1, N - 1] = -u[N - 2] / 6 + 2 * u[N - 1] / 3
        jac[N - 1, N - 2] = -u[N - 2] / 3 - u[N - 1] / 6
        for i in range(1, N - 1):
            jac[i, i - 1] = -u[i - 1] / 3 - u[i] / 6
            jac[i, i] = 1/6 * u[i + 1] - 1/6 * u[i - 1]
            jac[i, i + 1] = u[i] / 6 + u[i + 1] / 3
        return jac / h
    
    def _store_data(self, data, t, storage, write_adj_deps=False, write_ics=False):
        """Store the forward data.

        Parameters
        ----------
        data : array
            The forward data.
        t : int
            The time step.
        storage : StorageType
            The storage type.
        write_adj_deps : bool, optional
            If `True`, the adjoint dependency data will be stored.
        write_ics : bool, optional
            If `True`, the forward restart data will be stored.
        """
        if storage == StorageType.DISK:
            if write_adj_deps:
                self._store_on_disk(data, t, adj_deps=write_adj_deps)
            if write_ics:
                self._store_on_disk(data, t)
        elif storage == StorageType.RAM and write_adj_deps:
            self.adjoint_dependency[StorageType.RAM][t] = copy.deepcopy(data)

    def _store_on_disk(self, data, step, adj_deps=False):
        """Store the forward data on disk.

        Parameters
        ----------
        data : array
            The forward data.
        step : int
            The time step.
        adj_deps : bool, optional
            If True, the data is stored in the adjoint dependencies folder.
        """
        if adj_deps:
            file_name = "adj_deps/fwd_"+ str(step) +".npy"
            with open(file_name, "wb") as f:
                np.save(f, data)
        else:
            file_name = "fwd_data/fwd_"+ str(step) +".npy"
            with open(file_name, "wb") as f:
                np.save(f, data)
        self.snapshots[StorageType.DISK][step] = file_name

## Checkpointing
The adjoint PDE is then solved in a reverse time order and it depends as observed in Jacobian in Eq. (7). It is therefore necessary to have access to the relevant forward data. Storing the entire forward solution uses storage linear linear in the number of time steps, which can exhaust the available storage. To overcome this kind of problem, checkpointing algorithms are used to reduce the memory usage.

To employ a checkpointing method in the adjoint-based sensitivity calculation, we define a `CheckpointingManager` to manage the execution of forward and adjoint models. The `CheckpointingManager` defines the `execute` method, which performs each action specified in a provided checkpoint schedule (`_schedule`). Within the `execute` method, we have the single-dispatch generic `action` function that is overloaded by particular functions. For instance, if the `checkpoint_schedule` action is `Forward`, the `action_forward` function is called, and this can advance the forward calculation. In this particular example, the forward solver is executed by calling `self.equation.forward`. Here, `self.equation` is an attribute of `CheckpointingManager`. Similarly, the adjoint solver is executed by calling `self.equation.adjoint` within the `action_reverse` function.

In [4]:
import functools, sys
from checkpoint_schedules import *
class CheckpointingManager:
    """Manage the forward and adjoint solvers.

    Attributes
    ----------
    schedule : CheckpointSchedule
        The schedule created by `checkpoint_schedules` package.
    equation : object
        An equation object used to solve the forward and adjoint solvers.
    
    Notes
    -----
    The `equation` object contains methods to execute the forward and adjoint. In 
    addition, it contains methods to copy data from one storage to another, and
    to set the initial condition for the adjoint.
    """
    def __init__(self, schedule, equation):
        self.max_n = sys.maxsize
        self.equation = equation
        self.reverse_step = 0
        self._schedule = schedule
        self.single_storage = False
        if (
                isinstance(self._schedule, SingleMemoryStorageSchedule) 
                or isinstance(self._schedule, SingleDiskStorageSchedule)
        ):
            self.single_storage = True
        
    def execute(self):
        """Execute forward and adjoint using checkpointing.
        """
        @functools.singledispatch
        def action(cp_action):
            raise TypeError("Unexpected action")

        @action.register(Forward)
        def action_forward(cp_action):
            n1 = cp_action.n1
            self.equation.forward(cp_action.n0, n1, storage=cp_action.storage,
                                  write_adj_deps=cp_action.write_adj_deps,
                                  write_ics=cp_action.write_ics)
            if n1 >= self.equation.model["max_n"]:
                n1 = min(n1, self.equation.model["max_n"])
                self._schedule.finalize(n1)

        @action.register(Reverse)
        def action_reverse(cp_action):
            self.equation.adjoint(cp_action.n0, cp_action.n1, cp_action.clear_adj_deps)
            self.reverse_step += cp_action.n1 - cp_action.n0
            
        @action.register(Copy)
        def action_copy(cp_action):
            self.equation.copy_data(cp_action.n, cp_action.from_storage, cp_action.to_storage,
                                    move=False, single_storage=self.single_storage)

        @action.register(Move)
        def action_move(cp_action):
            self.equation.copy_data(cp_action.n, cp_action.from_storage, cp_action.to_storage,
                                    move=True, single_storage=self.single_storage)
            
        @action.register(EndForward)
        def action_end_forward(cp_action):
            if self._schedule.max_n is None:
                self._schedule._max_n = self.max_n
            
        @action.register(EndReverse)
        def action_end_reverse(cp_action):
            if self._schedule.max_n != self.reverse_step:
                raise ValueError("The number of steps in the reverse phase"
                                 "is different from the number of steps in the"
                                 "forward phase.")
            
        self.reverse_step = 0
        for _, cp_action in enumerate(self._schedule):
            action(cp_action)
            if isinstance(cp_action, EndReverse):
                break

## Adjoint-based sensitivity computations

The purpose of this section is to apply every checkpointing approach available in the `checkpoint_schedules` package and verify if every approach converges to the same gradient.

Below, we define the `model` dictionary containing the parameters required for the forward and adjoint solvers. The `model` dictionary is then passed to the `BurgersEquation` class. Additionally, we set up the 1D mesh and the initial condition for the forward Burgers' solver.

In [5]:
model = {"lx": 1,   # domain length
         "nx": 100,  # number of nodes
         "dt": 0.0001,  # time step
         "nu": 0.01,  # viscosity
         "max_n": 500,  # total steps
         "chk_ram": 50,  # number of checkpoints in RAM
         "chk_disk": 50,  # number of checkpoints on disk
        }
mesh = np.linspace(0, model["lx"], model["nx"]) # create the spatial grid
u0 = np.sin(np.pi*mesh) # initial condition

We start by computing the gradient without any checkpointing. Then, we compute the gradient using the `SingleMemoryStorageSchedule`, `SingleMemoryStorageSchedule`, `Revolve` and `HRevolve` checkpointing schedules. We then compare the gradients obtained from each checkpointing schedule to the gradient obtained without checkpointing. We expect the gradients to be the same. 

In [6]:
burger = BurgersEquation(model, u0, mesh)  # Create the burger's equation object.
burger.forward(0, model["max_n"], storage=StorageType.WORK, write_adj_deps=True)  # Solve the forward problem.
burger.adjoint(0, model["max_n"], clear_adj_deps=True)  # Solve the adjoint problem.
base_gradient = burger.gradient()  # Compute the gradient.

/Users/ddolci/work/my_venv/lib/python3.11/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


We now compute the adjoint-based gradient using `SingleMemoryStorageSchedule` checkpointing approach. The `SingleMemoryStorageSchedule` stores the forward data of each time-step in working memory. As explained in the [notebook with illustrative example](https://nbviewer.org/github/firedrakeproject/checkpoint_schedules/blob/main/docs/notebooks/tutorial.ipynb), this schedule does not require the maximal step (`model["max_n"]`).

In [7]:
schedule = SingleMemoryStorageSchedule()  # Create the checkpointing schedule.
burger = BurgersEquation(model, u0, mesh)  # Create the burger's equation object.
manager = CheckpointingManager(schedule, burger)  # Create the checkpointing manager.
manager.execute()  # execute the checkpointing schedule using `SingleMemoryStorageSchedule` schedule.
print("Max gradient difference: ", np.max(np.abs(base_gradient - burger.gradient())))  # Compute the gradient difference with the base gradient.


Max gradient difference:  0.0


The next example shows the usage of the `SingleDiskStorageSchedule` schedule with the `move_data=True` argument. In this case, the forward data used in the adjoint computations, and initially stored on disk, is removed from disk as the adjoint calculation progresses.

In [8]:
schedule = SingleDiskStorageSchedule(move_data=True) # Create the checkpointing schedule.
burger = BurgersEquation(model, u0, mesh)  # Create the burger's equation object.
manager = CheckpointingManager(schedule, burger)  # Create the checkpointing manager.
manager.execute()  # execute the checkpointing schedule using `SingleMemoryStorageSchedule` schedule.
print("Max gradient difference: ", np.max(np.abs(base_gradient - burger.gradient())))  # Compute the gradient difference with the base gradient.

Max gradient difference:  0.0


The following example uses `Revolve` schedule [1]. The `Revolve` algorithm requires the definition of the maximal step `model["max_n"]` before the execution of the forward solver, and also the specification of the number of checkpoints stored in memory.

In [9]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = Revolve(model["max_n"], model["chk_ram"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
print("Max gradient difference: ", np.max(np.abs(base_gradient - burger.gradient())))  # Compute the gradient difference with the base gradient.

Max gradient difference:  0.0


In [10]:
burger = BurgersEquation(model, u0, mesh) # create the burger's equation object
schedule = HRevolve(model["max_n"], model["chk_ram"], model["chk_disk"]) # create the checkpointing schedule
manager = CheckpointingManager(schedule, burger)  # create the checkpointing manager
manager.execute()  # execute the forward and adjoint solvers using checkpointing
print("Max gradient difference: ", np.max(np.abs(base_gradient - burger.gradient())))  # Compute the gradient difference with the base gradient.

Max gradient difference:  0.0


Feel free to explore alternative schedules provided by the `checkpoint_schedules` package. Our [notebook with illustrative example](https://nbviewer.org/github/firedrakeproject/checkpoint_schedules/blob/main/docs/notebooks/tutorial.ipynb) offers a demonstration of their usage. You need follow the steps outlined in the above code: instantiate a `BurgersEquation` object, define the `schedule`, create a `CheckpointingManager` object, and then execute the solvers using the `execute` method.

### References

[1] Griewank, A., & Walther, A. (2000). Algorithm 799: revolve: an implementation of checkpointing for the reverse or adjoint mode of computational differentiation. ACM Transactions on Mathematical Software (TOMS), 26(1), 19-45., doi: https://doi.org/10.1145/347837.347846